<div style="background-color: orange; padding: 10px; border-radius: 5px; text-align: center;box-shadow:8px 8px 8px 8px;color:white;">
  <h1 style="color: black;">Natural Entity Recoginition model (NER) </h1>
</div>

# Introduction:
This notebook demonstrates the implementation of a **Hybrid Natural Language Processing (NLP) Architecture** designed for high-precision entity extraction in e-commerce environments (e.g., brand names, specific SKUs).

#### We are building an AI model that:
- Reads a sentence word by word
- Understands the context of each word
- Assigns a label (tag) to every word

This task is called Named Entity Recognition (NER)

# 1. Imports

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import warnings
import os

#stop the tensorflow warnings since we run the model training on cpu
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

<div style="background-color: cyan; padding: 10px; border-radius: 5px; text-align: center;box-shadow:8px 8px 8px 8px;color:white;">
  <h1 style="color: black;">Preprocessing </h1>
</div>

# 2. Initalising the dataset

### Dataset:

The dataset was given in sentences to simulate given raw data for ai to proccess and train on

In [9]:

#These sentneces are for training the AI model
sentences = [
    "grab a black coat today in paris",
    "I would like a pink sneakers tomorrow in cairo",
    "show me a purple hat on friday in london",
    "looking for blue tshirt on wednesday in Egypt",

    "grab a yellow coat today",
    "I would like a gray sneakers tomorrow",
    "show me a white hat on friday",
    "looking for orange tshirt on wednesday",

    "grab a black coat in paris",
    "I would like a red sneakers in cairo",
    "show me a green hat in london",
    "looking for brown tshirt in Egypt",

    "give me a pink chocolate",
    "I would like a gold sneakers",
    "red hoodie is cool",
    "blue shirt for sale"

]

### Labeling the data set:

* 0 -> O (unknown)
* 1 -> color
* 2 -> Product
* 3 -> Date
* 4 -> Location

In [10]:


labels = [
    [0,0,1,2,3,0,4],
    [0,0,0,0,1,2,3,0,4],
    [0,0,0,1,2,0,3,0,4],
    [0,0,1,2,0,3,0,4],

    [0,0,1,2,3],
    [0,0,0,0,1,2,3],
    [0,0,0,1,2,0,3],
    [0,0,1,2,0,3],

    [0,0,1,2,0,4],
    [0,0,0,0,1,2,0,4],
    [0,0,0,1,2,0,4],
    [0,0,1,2,0,4],

    [0,0,0,1,2],
    [0,0,0,0,1,2],
    [1, 2, 0, 0],
    [1, 2, 0, 0]
]

# 3. Tokenizing

We setup a tokenizer to seperate the sentences into an array of words

In [11]:

# Setup Tokenizer
tokenizer = Tokenizer(oov_token="OOV")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # +1 for padding

# Convert Text to Sequences (Integers)
sequences = tokenizer.texts_to_sequences(sentences)

# Padding (Make all sentences same length for the Matrix Math)
max_length = 20
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
padded_labels = pad_sequences(labels, maxlen=max_length, padding='post')

# One-Hot Encode Labels for the Softmax Layer
# (0 -> [1,0,0], 1 -> [0,1,0], 2 -> [0,0,1])
Y = to_categorical(padded_labels, num_classes=5)

print(f" Data Ready. Vocab Size: {vocab_size}")

 Data Ready. Vocab Size: 44


<div style="background-color: cyan; padding: 10px; border-radius: 5px; text-align: center;box-shadow:8px 8px 8px 8px;color:white;">
  <h1 style="color: black;">Model training </h1>
</div>

# 4. Model Structure

The model is using the following layers:
* Embedding layers: Turns words into a dense vector of 16 numbers

* Bidirectional Layers:Understands the context of sentences by detecting if a label would likely come after another label

* Dropout: Stops a precentage of the neurons in the neural network to prevent overfitting

* Time distributed Dense Softmax layer: The classifier of the words using a softmax activation for multivariable logistic regression, TimeDistributed is used so the classifier applies for each words rather than entire sentence

In [12]:

model = tf.keras.Sequential([

    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=16, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(5, activation='softmax'))

])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("\n Training the model...")
model.fit(padded_sequences, Y, epochs=300, verbose=0)
print(" Training Complete.")



 Training the model...
 Training Complete.


<div style="background-color: cyan; padding: 10px; border-radius: 5px; text-align: center;box-shadow:8px 8px 8px 8px;color:white;">
  <h1 style="color: black;">Model Prediction </h1>
</div>

# 5. Model Prediction test

In [13]:
# --- STEP 5: FIELD TEST (Prediction) ---
print("\n🔮 PREDICTION TEST:")
test_sentence = ["I wear blue sneakers"]
test_seq = tokenizer.texts_to_sequences(test_sentence)
test_padded = pad_sequences(test_seq, maxlen=max_length, padding='post')

prediction = model.predict(test_padded)
predicted_ids = np.argmax(prediction, axis=-1)[0]

# Decode the Output
id_to_word = {v: k for k, v in word_index.items()}
tag_map = {
    0: "O",
    1: "COLOR",
    2: "PRODUCT",
    3: "DATE",
    4: "LOCATION"
}

print(f"📝 Input: {test_sentence[0]}")
print("-" * 30)
for i, word_id in enumerate(test_padded[0]):
    if word_id != 0: # Skip padding
        word = id_to_word[word_id]
        tag = tag_map[predicted_ids[i]]
        # Show the Probability confidence (Your "Stats" Brain)
        confidence = np.max(prediction[0][i])
        print(f"Word: '{word}' \t-> Tag: {tag} \t(Conf: {confidence:.2f})")


🔮 PREDICTION TEST:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
📝 Input: I wear blue sneakers
------------------------------
Word: 'i' 	-> Tag: O 	(Conf: 0.99)
Word: 'OOV' 	-> Tag: O 	(Conf: 0.62)
Word: 'blue' 	-> Tag: COLOR 	(Conf: 0.80)
Word: 'sneakers' 	-> Tag: PRODUCT 	(Conf: 0.92)


## Save model for deployment

In [14]:
model.save('ner_high_accuracy.h5')